In [4]:
!pip install gekko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 32.0 MB/s eta 0:00:0000:0100:01


In [5]:
from gekko import GEKKO
import numpy as np
import random

In [ ]:
# rep code decoder

# settings ----
num_qubits = 3
num_stabilizers = num_qubits - 1
p = 0.1

ps = {} # error probs
for i in range(num_qubits):
    ps[i] = p # constant prob of error for now

Cs = { # stabilizer check values, these are the measured stabilizer values
    (0,1): -1, # Parity check value: -1 
    (1,2): 1 # Parity check value: 1
}
# The above stabilizers indicate Z0Z1 = -1 and Z1Z2 = 1, so there's probably an error on the 0th qubit
# Let's see if the MLE decoder can figure this out

# set up gekko ----
m = GEKKO(remote=False)
m.options.SOLVER = 1  # APOPT is an MINLP solver
# optional solver settings with APOPT
m.solver_options = [
    "minlp_maximum_iterations 500",  # minlp iterations with integer solution
    "minlp_max_iter_with_int_sol 10",  # treat minlp as nlp
    "minlp_as_nlp 0",  # nlp sub-problem max iterations
    "nlp_maximum_iterations 50",  # 1 = depth first, 2 = breadth first
    "minlp_branch_method 1",  # maximum deviation from whole number
    "minlp_integer_tol 0.05",  # covergence tolerance
    "minlp_gap_tol 0.01",
]


# set up variables ---
Es = {}
for i in range(num_qubits):
    Es[i] = m.Var(
        value=random.randint(0,1), lb=0, ub=1, integer=True
    )

Ks = {}
for k in Cs.keys():
    Ks[k] = m.Var(
        value=random.randint(0,1), lb=0, integer=True
    )


# Objective ---
m.Obj(
    -m.sum(
        [
            np.log(ps[j]) * Es[j]  + np.log(1 - ps[j]) * (1 - Es[j])
            for j in range(num_qubits)
        ]
    )
)


# Constraints ---

for key, val in Cs.items():
    i, j = key
    m.Equation(Es[i] + Es[j] - 2 * Ks[key] == (1 - val)/2)

m.solve()

print("Solution (0: no error, 1: error): ", Es)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            1
   Constants    :            0
   Variables    :            9
   Intermediates:            0
   Connections  :            4
   Equations    :            6
   Residuals    :            6
 
 Number of state variables:              9
 Number of total equations: -            6
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              3
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    2 Dpth:    0 Lvs:    3 Obj:  2.51E+00 Gap:       NaN
Iter:     2 I: -1 Tm:      0.00 NLPi:    1 Dpth:    1 Lvs:    2 Obj:  2.51E